In [1]:
from graph import *
import os, json
import networkx as nx

from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Category20

savedir = "saves"
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
output_notebook()

def plot_bokeh(g, title):
    color_palette = ('#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a',
                     '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', 
                     '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', 
                     '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', 
                     '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', 
                     '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf',
                     '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', 
                     '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', 
                     '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf'
                    )
    degrees = dict(nx.degree(g.nx_graph))
    nx.set_node_attributes(g.nx_graph, name='degree', values=degrees)
    max_degree = max([c for x, c in g.in_degree.items()])

    adjusted_node_size = dict([(node, 5+(5*(degree/max_degree))) for node, degree in g.in_degree.items()])
    nx.set_node_attributes(g.nx_graph, name='adjusted_node_size', values=adjusted_node_size)

    modularity_class = {}
    modularity_color = {}
    for community_number, community in g.clusters.items():
        for name in community: 
            modularity_class[name] = community_number
            modularity_color[name] = color_palette[community_number]
    nx.set_node_attributes(g.nx_graph, modularity_class, 'modularity_class')
    nx.set_node_attributes(g.nx_graph, modularity_color, 'modularity_color')
    plot = figure(tools="pan,wheel_zoom,save,reset, tap", active_scroll='wheel_zoom',
                x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)
    network_graph = from_networkx(g.nx_graph, nx.spring_layout, scale=10, center=(0, 0))
    network_graph.node_renderer.glyph = Circle(size='adjusted_node_size', fill_color='modularity_color')
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1) # XXX weight?
    plot.renderers.append(network_graph)
    show(plot)

Loading BokehJS ...

In [5]:
# Play with graph construction here
g = Graph(num_nodes=300,
          num_cores=1,
          core_connectivity=1.0,
          add_nodes_random=0.5,
          add_nodes_popularity=1.0,
          popularity_cutoff=0.5,
          connect_cores_directly=0.0,
          connect_second_neighbours=1.0,
          move_nodes_second_neighbour=0.5,
          connect_random=0.1)
plot_bokeh(g, "graph")

In [6]:
conf = {"cores": [100, 100, 100],
        "core_core_connections": [[0, 1, 10], [1, 2, 10]],
        "core_core_connection_type": [["indirect"], ["indirect"]],
        "extra_nodes_random": [0, 0, 0],
        "extra_nodes_popularity": [50, 50, 50],
        "popularity_cutoff": [1.0, 1.0, 1.0],
        "extra_edges_second_neighbour": [0, 0, 0],
        "extra_edges_random": [0, 0, 0]}
g = Graph(config=conf)
g.print_basic_stats()
plot_bokeh(g, "graph")
g.write_gexf(os.path.join(savedir, "graph_odd.gexf"))

Nodes: 473 Edges: 493


In [7]:
conf = {"cores": [200],
        "core_intra_connections": [100],
        "core_core_connections": [],
        "core_core_connection_type": [],
        "extra_nodes_random": [100],
        "extra_nodes_popularity": [400],
        "popularity_cutoff": [-0.2],
        "extra_edges_second_neighbour": [100],
        "extra_edges_random": [10]}
g = Graph(config=conf)
g.print_basic_stats()
plot_bokeh(g, "graph")

Nodes: 701 Edges: 811
